<a href="https://colab.research.google.com/github/dylanesq/2-Months-Project-LLM/blob/main/LlamaIndex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install llama-index


# **python-docx2txt**
A pure python-based utility to extract text from docx files.

The code is taken and adapted from python-docx. It can however also extract text from header, footer and hyperlinks. It can now also extract images.

In [ ]:
pip install docx2txt

In [ ]:
from llama_index.readers.file.docs import DocxReader
docx_reader = DocxReader()
document_1 = docx_reader.load_data('/content/trial.docx')


Chunking Randomly
[Documentation to TokenTextSplitter](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.node_parser.TokenTextSplitter.html#tokentextsplitter)

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    separator=" ",
)

In [ ]:
token_nodes = splitter.get_nodes_from_documents(
    document_1, show_progress=True
)

In [ ]:
!huggingface-cli login

# LLM SETUP
*   [HuggingFaceLLM](https://docs.llamaindex.ai/en/v0.9.48/api_reference/llms/huggingface.html#huggingfacellm)
*   [Llama 3.2 3B
](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)



In [ ]:
pip install llama-index-llms-huggingface

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""


In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    tokenizer_name="meta-llama/Llama-3.2-3B-Instruct",
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    device_map="auto",
    # loading model in 8bit for reducing memory
    #model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

**Embedding & Vectorization**


Vector Imports

In [ ]:
from llama_index.core import VectorStoreIndex
#NOT REQUIRED SimpleDirectoryReader, ServiceContext  #Vector store index is for indexing the vector


Embedding Imports
* [HuggingFaceEmbeddings](https://docs.llamaindex.ai/en/stable/examples/embeddings/huggingface/)  
*   [LAngchain HF Embedding](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html#huggingfaceembeddings)
*   [SBERT](https://www.sbert.net/)
*   [Llamindex embedding](https://docs.llamaindex.ai/en/stable/api_reference/embeddings/huggingface/)





In [ ]:
!pip install llama-index-embeddings-huggingface


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Set up the HuggingFaceEmbedding class with the required model to use with llamaindex core.
embed_model  = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

#Service Context/Settings

In [ ]:
# from llama_index.core import ServiceContext this is obsolote
from llama_index.core import Settings

Set Up
Edit  : ServiceContext is deprecated.

*   Use llama_index.settings.Settings instead
*   [Migration to Settings Llama Index](https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/)


In [ ]:
Settings.llm=llm
Settings.embed_model=embed_model
Settings.node_parser=splitter

Indexing

In [ ]:

index=VectorStoreIndex.from_documents(document_1)#no longer required to have servicecontext
query_engine=index.as_query_engine()

#Query



*   [Querying LlamaIndex](https://docs.llamaindex.ai/en/stable/module_guides/querying/)
*   [Query Engine Module
](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/modules/)


Custom Query engine **[not working]**

In [ ]:
import llama_index
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor


In [ ]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])

**Vanilla Query Response**

In [ ]:
response=query_engine.query("what is this document?")
#response.source_nodes[0].node.get_text()

In [ ]:
print(response)